In [1]:
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol
import pandas as pd

In [2]:
structures_path = Path("../structures/irc_opt_tests")

mols = {}

for f in structures_path.rglob("*.xyz"):
    with open(f, "r") as file:
        xyz_block = file.read()
        mol = xyz2mol(xyz_block)
        mols[f.stem] = (mol, [0])

In [3]:
step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

In [4]:
dfs = []
for i in range(len(df0)):
    df_tmp = df0.iloc[[i]]
    dfs.append(df_tmp)

In [5]:
def run_irc(df, debug=False, n_cores=12, mem_gb=30):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    step = Stepper([name],
                step_type="IRC",
                debug=debug,
                save_output_dir=False,
                n_cores=n_cores,
                memory_gb=mem_gb)
    
    options = {
        "wB97X-D3" : None,
        "6-31G**"  : None,
        "TightSCF" : None,
        "Freq"     : None,
        "NoSym"    : None,
        "Opt"      : None,
    }

    df1 = step.orca(df, "DFT-IRC-my-opt", options, save_step=True)
    results_dir = Path("results_irc_opts")
    results_dir.mkdir(exist_ok=True)
    df1.to_parquet(f"{results_dir}/df_irc_opts_{name}.parquet")

In [6]:
import submitit

DEBUG           = False
N_CORES         = 24
MEM_GB          = 60
TIMEOUT_MIN     = 14400


executor = submitit.AutoExecutor("logs/irc_opts")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

for dfi in dfs:
    name = "IRC_OPTS_" + dfi["custom_name"].iloc[0] 
    executor.update_parameters(slurm_job_name=name)
    executor.submit(run_irc, dfi, DEBUG, N_CORES, MEM_GB)
    print(f"Submitted: {name}")

Submitted: IRC_OPTS_TS3_font_IRC_B
Submitted: IRC_OPTS_ts2_font_p2_IRC_F
Submitted: IRC_OPTS_ts2_font_p1_IRC_F
Submitted: IRC_OPTS_ts1_font_IRC_F
Submitted: IRC_OPTS_ts1_font_IRC_B
Submitted: IRC_OPTS_ts2_font_p2_IRC_B
Submitted: IRC_OPTS_ts2_font_p1_IRC_B
Submitted: IRC_OPTS_TS3_font_IRC_F
